In [8]:
HOPSWORK_PROJECT_NAME = "taxidemand001"

In [5]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

load_dotenv(PARENT_DIR / ".env")

HOPSWORK_API_KEY = os.environ["HOPSWORK_API_KEY"]

In [7]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year + 1):
    rides_one_year = load_raw_data(year)
    rides = pd.concat([rides, rides_one_year])

 File 2022-01 already exists
 File 2022-02 already exists
 File 2022-03 already exists
 File 2022-04 already exists
 File 2022-05 already exists
 File 2022-06 already exists
 File 2022-07 already exists
 File 2022-08 already exists
 File 2022-09 already exists
 File 2022-10 already exists
 File 2022-11 already exists
 File 2022-12 already exists
Failed to download 2024-11
Failed to download 2024-12


In [10]:
rides

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
...,...,...
3833766,2024-10-31 23:49:01,107
3833767,2024-10-31 23:35:15,137
3833768,2024-10-31 23:30:43,188
3833769,2024-10-31 23:00:00,230
